# Build Net

In [8]:
import os
from bni_netica.bni_netica import Net
from benchmarking.data_generator import build_random_bn
from bn_helpers.support_tools import get_nets, printNet
from benchmarking.data_utils import save_nets_to_parquet, load_nets_from_parquet
from benchmarking.benchmarking_utils import pickTwoRandomNodes
from bn_helpers.support_tools import get_BN_structure, printNet
from bn_helpers.bn_helpers import AnswerStructure, BnHelper
from ollama.prompt import answer_this_prompt


In [2]:
output_path = "./nets/outputs/"

net3 = build_random_bn(
    n_nodes=12,
    name="SkewBN",
    cpt_mode="random",
    dirichlet_alpha=0.3,     # spiky rows
    avg_edges_per_node=1.4,
    max_in_degree=2,
    sprinkle_motifs=5,
    # save_path=output_path
)

printNet(net3)

A -> ['F']
E -> ['L', 'F']
F -> ['D']
D -> []
L -> []
I -> ['C', 'H']
B -> ['K', 'J']
J -> ['C']
C -> []
G -> ['K']
K -> ['H']
H -> []


## Test the boundary

In [3]:
net2 = build_random_bn(
    n_nodes=1000,
    name="MaxRandomBN",
    cpt_mode="random",
    avg_edges_per_node=2,
    max_in_degree=2,
    sprinkle_motifs=5,
    # save_path=output_path
)

# printNet(net2)


# Build Dataset

In [5]:
# import all nets from the collection
nets_collection = get_nets()

# print all nets
for net in nets_collection:
    printNet(net)
    print()

A -> ['B', 'C']
B -> ['D']
C -> ['D', 'E']
D -> []
E -> []

VisitAsia -> ['Tuberculosis']
Tuberculosis -> ['TbOrCa']
Smoking -> ['Cancer', 'Bronchitis']
Cancer -> ['TbOrCa']
TbOrCa -> ['XRay', 'Dyspnea']
XRay -> []
Bronchitis -> ['Dyspnea']
Dyspnea -> []

Class -> ['Obsv', 'Result']
Obsv -> ['Classifier']
Classifier -> ['Result']
Result -> []

Sex -> ['Smoking_status', 'LVH', 'Coronary_artery_disease']
Smoking_status -> ['Coronary_artery_disease']
Systolic_Blood_Pressure -> ['LVH', 'Coronary_artery_disease']
Age -> ['LVH', 'Coronary_artery_disease']
LVH -> ['Coronary_artery_disease']
Diabetes_mellitus -> ['Coronary_artery_disease']
Total_chol -> ['Coronary_artery_disease']
HDL_Status -> ['Coronary_artery_disease']
Coronary_artery_disease -> []

Tampering -> ['Alarm']
Fire -> ['Alarm', 'Smoke']
Alarm -> ['Leaving']
Leaving -> ['Report']
Smoke -> []
Report -> []

P1 -> ['C', 'Color_P1']
P2 -> ['C', 'Color_P2']
C -> ['Color_C']
Color_P1 -> []
Color_C -> []
Color_P2 -> []

Ecstazine -> ['N

In [1]:
data_output = "./benchmarking/data"

In [ ]:
# Generate 500 random nets
nets = nets_collection + \
[ 
    build_random_bn(n_nodes=i,
        cpt_mode="random",
        avg_edges_per_node=2,
        max_in_degree=2,
        sprinkle_motifs=5
    )
    for i in range(3,800)
]

# Save them
save_nets_to_parquet(nets, os.path.join(data_output, "nets_dataset_800.parquet"))

In [4]:
LOAD_NETS = True
# Load them back
if LOAD_NETS:
    loaded_nets = load_nets_from_parquet(os.path.join(data_output, "nets_dataset.parquet"))


Loaded 509 nets from ./benchmarking/data/nets_dataset.parquet


In [5]:
for net in loaded_nets[:20]:
  printNet(net)
  print()

A -> ['B', 'C']
B -> ['D']
C -> ['D', 'E']
D -> []
E -> []

VisitAsia -> ['Tuberculosis']
Tuberculosis -> ['TbOrCa']
Smoking -> ['Cancer', 'Bronchitis']
Cancer -> ['TbOrCa']
TbOrCa -> ['XRay', 'Dyspnea']
XRay -> []
Bronchitis -> ['Dyspnea']
Dyspnea -> []

Class -> ['Obsv', 'Result']
Obsv -> ['Classifier']
Classifier -> ['Result']
Result -> []

Sex -> ['Smoking_status', 'LVH', 'Coronary_artery_disease']
Smoking_status -> ['Coronary_artery_disease']
Systolic_Blood_Pressure -> ['LVH', 'Coronary_artery_disease']
Age -> ['LVH', 'Coronary_artery_disease']
LVH -> ['Coronary_artery_disease']
Diabetes_mellitus -> ['Coronary_artery_disease']
Total_chol -> ['Coronary_artery_disease']
HDL_Status -> ['Coronary_artery_disease']
Coronary_artery_disease -> []

Tampering -> ['Alarm']
Fire -> ['Alarm', 'Smoke']
Alarm -> ['Leaving']
Leaving -> ['Report']
Smoke -> []
Report -> []

P1 -> ['C', 'Color_P1']
P2 -> ['C', 'Color_P2']
C -> ['Color_C']
Color_P1 -> []
Color_C -> []
Color_P2 -> []

Ecstazine -> ['N

# Benchmark simple Query: Is D-connected?

In [16]:
def get_ground_truth(net, node1, node2):
    bn_helper = BnHelper(function_name='is_XY_dconnected')

    ans = bn_helper.is_XY_dconnected(net, node1, node2)
    if ans:
        template = f"Yes, {node1} is d-connected to {node2}, which means that entering evidence for {node1} would change the probability of {node2} and vice versa."
    else:
        template = f"No, {node1} is not d-connected to {node2}, which means that entering evidence for {node1} would not change the probability of {node2}."

    return template


def benchmark_simple_query(net):
    node1, node2 = pickTwoRandomNodes(net)
    print('Two random nodes:', node1, node2)
    bn_helper = BnHelper(function_name='is_XY_dconnected')
    bn = get_BN_structure(net)
    prompt = f"In this Bayesian Network:\n{bn}\n"
    prompt += f"Is changing the evidence of {node1} going to change the probability of {node2}?"

    print(get_ground_truth(net, node1, node2))
    # ans = answer_this_prompt(prompt, format=AnswerStructure.model_json_schema())
    # ans = AnswerStructure.model_validate_json(ans)
    # return ans

In [17]:
print(benchmark_simple_query(loaded_nets[0]))

Two random nodes: C D
Yes, C is d-connected to D, which means that entering evidence for C would change the probability of D and vice versa.
None
